### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Pre-Process the data
### Drop Irrelevant feature
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

## Encode Categorical Variable
label_encoder=LabelEncoder()
data['Gender']=label_encoder.fit_transform(data['Gender'])

## One hot encoding geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder=OneHotEncoder()
geo_encoded_value=onehot_encoder.fit_transform(data[['Geography']])

##Converting into dataframe
geo_encoded_df=pd.DataFrame(geo_encoded_value.toarray(),columns=onehot_encoder.get_feature_names_out(['Geography']))

##Combine one hot encoded column into data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [4]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
X=data.drop("Exited",axis=1)
y=data["Exited"]

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


## Save the encoders and scalers
with open('label_encoder_gender1.pkl','wb') as file:
    pickle.dump(label_encoder,file=file)

with open('onehot_encoder_geo1.pkl','wb') as file:
    pickle.dump(onehot_encoder,file)

with open('sscaler1.pkl','wb') as file:
    pickle.dump(scaler,file)

In [17]:
## Define a function to create model and try different parameters (Keras Classifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for x in range(layers-1):
        model.add(Dense(neurons,activation="relu"))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model


In [18]:
## Create a Keras Classifier
model=KerasClassifier(layers=1,neurons=32, build_fn=create_model,epochs=50,batch_size=10, verbose=0)


In [19]:
## define grid search parameters

param_grid={
    'neurons':[15,32,64,128],
    'layers':[1,2,3],
    'epochs':[50,100]
}

In [21]:
## Perform grid search    cv=cross validation
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3,verbose=1)
grid_result=grid.fit(X_train,y_train)

##Print best parameters
print("Best : %f using %s " % (grid_result.best_score_,grid_result.best_params_))

Fitting 3 folds for each of 24 candidates, totalling 72 fits


c:\Users\Dell\Desktop\Python\DS\Deep learning Project\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Dell\Desktop\Python\DS\Deep learning Project\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best : 0.858249 using {'epochs': 100, 'layers': 1, 'neurons': 15} 
